In [6]:
data = {'L0 Domain': 'L0 Domain',
 'L1 Category': 'L1 Category',
 'L2 Category': 'L2 Category',
 'L3 Sub-Category': 'L3 Sub-Category',
 'L4 Product Enum-Code': 'L4 Product Enum-Code',
 'Product Name': 'Product Name',
 'Product Quantity': 'Product Quantity',
 'UOM': 'UOM',
 'MRP': 'MRP',
 'Price': 'Price',
 'Product Description': 'Product Description',
 'SKU Code': 'SKU Code',
 'Manufacturer': 'Manufacturer',
 'Country Of Origin': 'Country Of Origin',
 'Customer Care Contact': 'Customer Care Contact',
 'Time to Ship': 'Time to Ship',
 'Returnable (Y/N)': 'Returnable (Y/N)',
 'Cancellable (Y/N)': 'Cancellable (Y/N)',
 'COD available (Y/N)': 'COD available (Y/N)',
 'Images': 'Images',
 'Net Quantity': 'Net Quantity',
 'Brand': 'Brand',
 'Pack Quantity': 'Pack Quantity',
 'Pack Size': 'Pack Size',
 'Images / Video': 'Images / Video',
 'UPC/EAN': 'UPC/EAN',
 'FSSAI no': 'FSSAI no',
 'Preservatives (Y/N)': 'Preservatives (Y/N)',
 'Preservatives (details)': 'Preservatives (details)',
 'Flavours & Spices': 'Flavours & Spices',
 'Ready to cook (Y/N)': 'Ready to cook (Y/N)',
 'Ready to eat (Y/N)': 'Ready to eat (Y/N)',
 'Rice grain type': 'Rice grain type',
 'Recommended Age': 'Recommended Age',
 'Scented/ Flavour': 'Scented/ Flavour',
 'Herbal/ Ayurvedic': 'Herbal/ Ayurvedic',
 'Theme/ Occasion Type': 'Theme/ Occasion Type',
 'Hair Type': 'Hair Type',
 'Mineral Source': 'Mineral Source',
 'Caffeine Content': 'Caffeine Content',
 'Capacity': 'Capacity',
 'Composition': 'Composition',
 'Benefits': 'Benefits',
 'Usage': 'Usage',
 'Other details': 'Other details',
 'Marketed By': 'Marketed By',
 'Images 3': 'Images 3',
 'Images 4': 'Images 4',
 'Images 5': 'Images 5',
 'Images 6': 'Images 6',
 'Images 7': 'Images 7',
 'Images 8': 'Images 8',
 'Images 9': 'Images 9',
 'Baby Weight': 'Baby Weight',
 'Absorption Duration (in Hrs)': 'Absorption Duration (in Hrs)',
 'Features': 'Features',
 'Images 10': 'Images 10',
 'Care Instruction': 'Care Instruction',
 'Ingredients': 'Ingredients',
 'Specification': 'Specification',
 'Package Contains': 'Package Contains',
 'About': 'About'}

from enum import Enum

Columns = Enum('Columns', data)

In [7]:
from pydantic import BaseModel
from typing import List, Optional, Dict

class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """

    related_columns: List[Columns]

In [8]:
import os
from dotenv import load_dotenv

load_dotenv(
    "../ops/.env"
)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


import instructor
from openai import OpenAI

In [25]:
with open("../prompts/column_identifier_prompt.txt", "r") as file:
    column_prompt = file.read()

with open("../prompts/generate_sql_prompt.txt", "r") as file:
    sql_prompt = file.read()

In [10]:
client = instructor.patch(OpenAI())


def multi_classify(data: str) -> MultiClassPrediction:
    """Perform multi-label classification on the input text."""
    return client.chat.completions.create(
        model="gpt-4",
        response_model=MultiClassPrediction,
        messages=[
            {
                "role": "user",
                "content": f"{column_prompt} {data}",
            },
        ],
    )  # type: ignore

In [28]:
result = multi_classify("All products should have a product_id, product_name, product_description, and other_details")
result.model_dump_json()

'{"related_columns":["Product Name","Product Description","Other details"]}'

In [ ]:
import json

class SQLQUERYMODEL(BaseModel):
    """
    Class for an SQL QUERY 
    """
    queries: List[str]

def generate_sql_queries(requirement, column_names, table_name, sql_prompt, client: OpenAI, model: str = "gpt-4"):
    """
    Extracts the columns from the requirement
    """
    sql_prompt = sql_prompt.format(
        requirement=requirement, 
        column_names=column_names, 
        table_name=table_name
    )
    response = client.chat.completions.create(
        model=model,
        response_model=SQLQUERYMODEL,
        messages=[
            {
                "role": "user",
                "content": sql_prompt,
            },
        ],
    )
    return json.loads(response.model_dump_json())
    
generate_sql_queries("All products should have four image columns", "Images 1, Images 2, Images 3, Images 4", "products", sql_prompt, client)